In [1]:
!mkdir vikas

In [2]:
!cd vikas

In [3]:
!docker login -u vikasyetintala07@gmail.com -p ammananna

'docker' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
%%writefile config.py
bucket_name=""
bucket_uri="s3:"+"//"+bucket_name
version=1
artifact_uri=bucket_uri+str(version)
data_path=bucket_uri+"/"+"data/data.csv"
processed_train=bucket_uri+"/"+"processed/train.csv"
processed_test=bucket_uri+"/"+"processed/test.csv"

Writing config.py


In [5]:
%%writefile Dockerfile
FROM 
RUN apt-get update && apt-get install -y git
RUN mkdir pipeline
WORKDIR /pipeline
COPY ./ ./
RUN pip install -r requirements.txt

Writing Dockerfile


In [ ]:
!docker build -t vikashub/kubeflowsdk

In [ ]:
!docker push vikashub/kubeflowsdk

In [ ]:
!pip install kfp

In [13]:
import kfp
from kfp import components
from kfp import dsl

In [ ]:
@dsl.component
def get_data():
    getdata_component=kfp.dsl.ContainerOp(
        name="download-data",
        image="docker.io/vikashub/kubeflowsdk",
        command=["python","get_data.py"])
    return getdata_component

In [ ]:
@dsl.component
def process_data():
    data_processing=kfp.dsl.ContainerOp(
        name="data-processing",
        image="docker.io/vikashub/kubeflowsdk",
        command=["pyhton","data_process.py"])
    return data_processing

In [ ]:
@dsl.component
def training():
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='training',
        image='docker.io/vikashub/kubeflowsdk',
        command=['python', 'train.py'],
        file_outputs={'mlpipeline-ui-metadata':'artifacts/mlpipeline-ui-metadata.json', "mlpipeline-metrics":'artifacts/mlpipeline-metrics.json'}
        )
    
    return training_component

In [ ]:
debug=True
if debug:
    components=training()
    print(components)

In [ ]:
@dsl.pipeline(name="creating pipeline for predicting wine quality",
              description="it is an end to end pipeline")
def wine_pipeline():
    download_data=get_data()
    download_data.execution_options.caching_strategy.max_cache_staleness="P0D"
    data_processing=process_data().after(download_data)
    data_processing.execution_options.caching_strategy.max_cache_staleness="P0D"
    training=training().after(data_processing)
    training.execution_options.caching_strategy.max_cache_staleness="P0D"

In [ ]:
debug=True
if debug:
    components=wine_pipeline()
    print(components)

In [ ]:
kfp.compiler.Compiler.compile(wine_pipeline,'wine-pipeline-kubeflow.yaml')

In [ ]:
client=kfp.Clinet()

In [ ]:
EXPERIMENT_NAME="vikas_kubeflow"
experiment=client.create_experiment(name=EXPERIMENT_NAME)

In [ ]:
run=client.run_pipeline(experiment.id,"vikas_run","wine-pipeline-kubeflow.yaml")